# PDE  Week 13

Intro to PDE

Scipy with PDE

Most of the content of this Jupyter notebook has been taken from http://hplgit.github.io/prog4comp/doc/pub/p4c-sphinx-Python/._pylight006.html

When there is spatial and temporal dependence, the transient model is often a partial differential equation (PDE). Here we will give a glimse on how to proceed, if you have to solve PDE in your future.

We want to solve equations of the form

$$
\frac{\partial \Psi}{\partial t} = - C \nabla^2 \Psi + V(x,t) 
$$

As this is very complicated and we just want to go over with a simple introduction, let us consider (i) a 1D case (ii) $V(x,t)=g$. 

Therefore our equations looks like (in a very simplied version and with a heat source of $h(x,t)$)

$$
\frac{\partial \Psi}{\partial t} = \alpha \frac{d^2}{dx^2} \Psi + h(x,t)
$$

This is the so called Diffusion Equation, where $\Psi(x,t)$ can be the particle concentration in a liquid or other diffusion process. One very popular application of the diffusion equation is for heat transport in solid bodies. Here, $\Psi$ will correspond to the temperature, and the equation predicts how the temperature evolves in space and time within the solid body. We remark that the temperature in a fluid is influenced not only by diffusion, but also by the flow of the liquid. If present, the latter effect requires an extra term in the equation (known as an advection or convection term). The term g is known as the source term and represents generation, or loss, of heat (by some mechanism) within the body. For diffusive transport, g models injection or extraction of the substance.



## Initial Conditions and Boundary Conditions

The first thing, we need to worry is that the PDE needs to be solved in some given domain (in space and time). Therefore, we need to know the initial conditions and the boundary conditions (the solution is not unique unless we define both). For example for the diffusion equation we need to know the initial condition for the concentration $\Psi(x,0)$, stating what $\Psi$ is when the process starts. In addition, the diffusion equation needs one boundary condition at each point of the boundary within the predefined domain. This condition can either be that $\Psi$ is known or that we know the normal derivative, $\nabla \Psi \cdot \hat{n} = \partial \Psi/\partial \hat{n}$, where $\hat{n}$ denotes an outward unit normal the boundary of the domain.

Let us consider a very simple example of the temperature diffusion on a 1D rod with dimension $L$ (domain is given by $[0,L]$. On one end, we assume that the "rod" is insulating and on the other end, there is asome device that control the temperature of the medium. The temperature control is given by the function $c(t)$, let say at $x=0$. Therefore, we have then that $\Psi(0,t)=c(t)$ and $\partial \Psi(x=L,t)/\partial x = 0$ (no heat flux at $x=L$. 

Matematically, we want to solve

$$
\frac{\partial \Psi}{\partial t} = \alpha \frac{d^2}{dx^2} \Psi + h(x,t), \;\;\; x \in (0,L), \;\; t \in (0,T) \\
\Psi(0,t) = c(t), \;\;\; t \in (0,T)
\\
\frac{\partial \Psi (L,t)}{\partial t} = 0, \;\;\;  t \in (0,T)
\\
\Psi(x,0)=\phi(x), \;\;\; x \in (0,L)
$$




Now, we need to solve these equations, meaning that we need to evolve in time within a grid in space

## Finite Difference Methods

Let start by defining a evenly space grid in real space as $x_0 = 0 < x_1 < x_2 < \cdots < x_N = L$ ($x_{i+1} - x_i = \Delta x$).

Therefore, now we can use finite differences for the spatial dependence part:

$$
\frac{d^2 \Psi}{dx^2} \approx \frac{\Psi(x_{i+1},t) - 2 \Psi(x_i,t) + \Psi(x_{i-1},t)}{\Delta x^2}
$$

and following the notation we have used before, this is equivalent to

$$
\frac{\partial \Psi_i(t)}{\partial t} = \alpha \frac{\Psi_{i+1}(t) - 2 \Psi_i(t) + \Psi_{i-1}(t)}{\Delta x^2} + h_i(t) \;\;\;\;(1)
$$
Now, if you look carefully, here we have and $N-1$ ordinary differential equations, which we know how to solve (see our jupyter ODE notebook). This method, it is called the method of lines.

The only thing missing for this method to work is that we use the initial and boundary conditions we have defined.

$\Psi_i(t=0) = \phi_i$, which provides the initial condition to each equation.

For the boundary condition, we make the following observation, as $\Psi(0,t) = c(t)$, this is equivalent to $d\Psi(0,t)/dt = dc(t)/dt$, therefore $d\Psi_0/dt=dc(t)/dt$ with $\Psi_0(t=0)=c(t=0)$.  We use this equation instead of the approximated one from finite differences. 

The last thing to consider is the boundary at $x=L$. To to that, we make the following approximation (a centered approximation, which has a smaller error than the forward approximation):

$$
\frac{\partial \Psi_N}{dx} \approx \frac{\Psi_{N+1}-\Psi_{N-1}}{2 \Delta x} = 0
$$

In the case that you have other boundary, we can generalize the previouys equations to

$$
\frac{\partial \Psi_N}{dx} \approx \frac{\Psi_{N+1}-\Psi_{N-1}}{2 \Delta x} = \beta \\
\Psi_{N+1} =  2 \Delta x \beta + \Psi_{N-1} 
$$

NOTE: that the previous approximation implies the use of a "ficticious" point outside the domain. Here, we will use equation (1) for $i=N$ with the approximation $\Psi_{N+1} = \Psi_{N-1}$. Therefore we get for the case where it is equal to zero that

$$
\frac{\partial \Psi_N(t)}{\partial t} = \alpha \frac{2 \Psi_{N-1}(t) - 2 \Psi_N(t) }{\Delta x^2} + h_N(t) \;\;\;\;(2)
$$

We can now summarize all set of equations we need to solve:

$$
\frac{d\Psi_0}{dt}=\frac{dc(t)}{dt} \\
\frac{\partial \Psi_i(t)}{\partial t} = \alpha \frac{\Psi_{i+1}(t) - 2 \Psi_i(t) + \Psi_{i-1}(t)}{\Delta x^2} + h_i(t), \;\;\; i=1,\cdots,N-1 \\
\frac{\partial \Psi_N(t)}{\partial t} = \alpha \frac{2 \Psi_{N-1}(t) - 2 \Psi_N(t) }{\Delta x^2} + h_N(t)
\Psi_0(0) = c(0)\\
\Psi)_i(0) = \phi_i \\
$$


Ok Now, let us use this methodology for a small problem.

Let us take

$$
g(x,t)=3(x−L)\\
c(t)=−L(3t+2)\\
\phi(x)=2(x−L)
$$

In [5]:
from numpy import linspace, zeros, asarray
import matplotlib.pyplot as plt

#Useful functions
def ode_FE(f, U_0, dt, T):
    N_t = int(round(float(T)/dt))
    # Ensure that any list/tuple returned from f_ is wrapped as array
    f_ = lambda u, t: asarray(f(u, t))
    u = zeros((N_t+1, len(U_0)))
    t = linspace(0, N_t*dt, len(u))
    u[0] = U_0
    for n in range(N_t):
        u[n+1] = u[n] + dt*f_(u[n], t[n])
    return u, t

In [1]:
def rhs(u, t):
    N = len(u) - 1
    rhs = zeros(N+1)
    rhs[0] = dsdt(t)
    for i in range(1, N):
        rhs[i] = (beta/dx**2)*(u[i+1] - 2*u[i] + u[i-1]) + \
                 g(x[i], t)
    rhs[N] = (beta/dx**2)*(2*u[N-1] + 2*dx*dudx(t) -
                           2*u[N]) + g(x[N], t)
    return rhs

def u_exact(x, t):
    return (3*t + 2)*(x - L)

def dudx(t):
    return (3*t + 2)

def s(t):
    return u_exact(0, t)

def dsdt(t):
    return 3*(-L)

def g(x, t):
    return 3*(x-L)

In [4]:
def test_diffusion_exact_linear():
    global beta, dx, L, x  # needed in rhs
    L = 1.5
    beta = 0.5
    N = 4
    x = linspace(0, L, N+1)
    dx = x[1] - x[0]
    u = zeros(N+1)

    U_0 = zeros(N+1)
    U_0[0] = s(0)
    U_0[1:] = u_exact(x[1:], 0)
    dt = 0.1
    print(dt)

    u, t = ode_FE(rhs, U_0, dt, T=1.2)

    tol = 1E-12
    for i in range(0, u.shape[0]):
        diff = abs(u_exact(x, t[i]) - u[i,:]).max()
        assert diff < tol, 'diff=%.16g' % diff
        print('diff=%g at t=%g' % (diff, t[i]))